In [ ]:
#===============================================================================================
# Huggingface와 ONNX 런타임을 이용한 동적 양자화 예시 4
# => 훈련된 sbert 모델을 가지고, 최적화->양자화 과정을 거처 ONNX 모델로 로 만든 후, 테스트 하는 예시
# => sbert는 embedding 모델이므로 ORTModelForFeatureExtraction 이용
#
## 참고 : https://huggingface.co/blog/optimum-inference
##
## 아래 패키지들을 설치해야 함
'''
pip install datasets
pip install optimum
pip install optimum[onnxruntime]
pip install optimum[onnxruntime-gpu]  #gpu 사용인 경우
pip install transformers[onnx]
'''    

## 
# cannot import name 'FeaturesManager' from 'transformers.onnx' 에러나면 
# 아래처럼 transformers와 optimum 등을 업데이트 해줘야함.
# - 업데이트 하면 transformers 버전이 4.15에서 4.21.2 버전으로 업데이트 됨.
#
'''
pip uninstall transformers
pip install transformers
'''
# 혹은
'''
python -m pip install git+https://github.com/huggingface/optimum.git
python -m pip install git+https://github.com/huggingface/optimum.git#egg=optimum[onnxruntime]
'''
#참고: https://github.com/huggingface/optimum
#
# 이때 trnasformers를 업데이트 하고 나면 아래와 같은 모듈들도 모두 uninstall 하고 나서 최신으로 install 해줘야 함.
# - 아래 외에도 몇가지 더 있을수 있음. (*코드 수행하면서 필요한 경우 삭제후 재설치 해줘야 함)
'''
pip uninstall optimum[onnxruntime]
pip uninstall optimum[onnxruntime-gpu]  #gpu 사용인 경우
pip uninstall transformers[onnx]
pip uninstall optimum
pip uninstall huggingface-hub
pip uninstall pyyaml
pip uninstall tqdm
pip uninstall tokenizers
pip uninstall accelerate
pip uninstall regex
pip uninstall nltk
pip uninstall filelock
pip uninstall click
'''
#
#===============================================================================================

import numpy as np
import pandas as pd
import torch
import os
import torch.nn.functional as F
import sys
from transformers import AutoTokenizer
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging
logger = mlogging(loggername="sbert-optimized", logfilename="sbert-optimized")
seed_everything(111)


model_checkpoint = "bongsoo/klue-sbert-v1"

In [ ]:
import transformers
transformers.__version__

In [ ]:
# 1. ONNX 모델로 변환 
# => huggingface ORTModelForSequenceClassification 를 이용하여, ONNX 모델로 쉽게 저장할수 있다.(*단 여기서 ONNX 모델은 양자화된 모델은 아니고, 형식만 변경한 것임)
# => 양자화 하려면. 위에서 처럼 ORTQuantizer 이용해야 함.

from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer


# Load model from hub and export it through the ONNX format 
model = ORTModelForFeatureExtraction.from_pretrained(
    model_checkpoint, 
    from_transformers=True
)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the exported model
onnx_path = './onnxfolder'
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

In [ ]:
#2. 최적화 적용
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# optimization 최적화 실행
# optimization_config=99 enables all available graph optimisations(99이면 모든 것을 최적화 시킴)
optimization_config = OptimizationConfig(optimization_level=99)

optimizer = ORTOptimizer.from_pretrained(model_checkpoint, feature="default")

optimizer.export(
    onnx_model_path='./onnxfolder/model.onnx',   # 앞에서 만든 ONNX 모델 경로
    onnx_optimized_model_output_path='./onnxfolder/model-optimized.onnx',  # 새롭게 만들 optimized 모델 경로
    optimization_config=optimization_config,
)

In [ ]:
# 3. 양자화 
from optimum.onnxruntime import ORTConfig, ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig


# 동적 양자화인 경우 is_static = False로 해야 함.
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)

# 분류 모델인 경우에는 feature="sequence-classification"
quantizer = ORTQuantizer.from_pretrained(model_checkpoint, feature="default")

# ONNX 모델로 만들고 양자화 함
quantizer.export(
    onnx_model_path='./onnxfolder/model-optimized.onnx',   # optimized 모델 경로
    onnx_quantized_model_output_path="./onnxfolder/model-quantized.onnx",  
    quantization_config=qconfig,
)


In [ ]:
# onnxfoloder 에 양자화된 model-quantized.onnx 모델이름 변경
# => model-quantized.onnx 모델명을 model.onnx 로 변경 (*기존 model.onnx는 model_org.onnx로 이름 변경)

In [ ]:
#============================================================
# 양자화 모델 불러옴
# => 양자화 모델은 model.eval() 하면 에러남.
#============================================================

from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction, ORTModelForSequenceClassification

vocab_path = "bongsoo/klue-sbert-v1-onnx"
#vocab_path = "../../data11/model/onnx/klue-sbert-v1-onnx"   # model.onnx와 vocab파일이 있는 폴더 지정
model_path = vocab_path

tokenizer = AutoTokenizer.from_pretrained(vocab_path)

# 분류모델이면, ORTModelForSequenceClassification
# 문장임베딩이면, ORTModelForFeatureExtraction 호출
model = ORTModelForFeatureExtraction.from_pretrained(model_path)

print(model)

In [ ]:
# korsts 로딩
test_file1 = '../../data11/korpora/korsts/tune_test.tsv'

sentence1 = []
sentence2 = []
scores = [] 
    
with open(test_file1, 'rt', encoding='utf-8') as fIn1:
    lines = fIn1.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
            
        sentence1.append(s1)
        sentence2.append(s2)
        scores.append(score)
        
print(f'sentence1: {len(sentence1)}')
print(f'sentence2: {len(sentence2)}')
print(f'scores: {len(scores)}')

print(f'sentence1: {sentence1[0]}')
print(f'sentence2: {sentence2[0]}')
print(f'scores: {scores[0]}')

In [ ]:
# sentence1 + sentence2를 묶어서 tokenizer 처리함
corpus = sentence1 + sentence2
print(len(corpus))

corpus_inputs = tokenizer(corpus, 
                 add_special_tokens=True, 
                 truncation=True, 
                 padding=True,   
                 max_length=128, 
                 return_tensors="pt")
print(corpus_inputs)
print(corpus_inputs['input_ids'])
print(f'type:{type(corpus_inputs)}')
print(corpus_inputs['input_ids'].size())


In [ ]:
# embedding 값 구하기 
outputs = model(**corpus_inputs)
embedding = outputs.last_hidden_state
print(f'embed_len:{embedding.shape}')

In [ ]:
# 구한 embeding 값을 sentence1, sentence2 로 나눔.
embed_len = len(embedding)//2
print(embed_len)
tembed1 = embedding[0:embed_len]
tembed2 = embedding[embed_len:]

print(tembed1.shape)
print(tembed2.shape)

# embed1,2는 3차원->1차원으로 만들어야 함.
# => 평균값으로 함
embedlist1 = []
for idx,embedding in enumerate(tembed1): # enumerate는 index, value 값이 리턴됨
    embed = torch.mean(embedding, dim=0).numpy()
    embedlist1.append(embed)

embedlist2 = []
for idx,embedding in enumerate(tembed2): # enumerate는 index, value 값이 리턴됨
    embed = torch.mean(embedding, dim=0).numpy()
    embedlist2.append(embed)


print(len(embedlist1))
print(len(embedlist2))

print(embedlist1[0].shape)
print(embedlist2[0].shape)

In [ ]:
# sklearn 을 이용하여 cosine_scores를 구함
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
cosine_scores = 1 - (paired_cosine_distances(embedlist1, embedlist2))

print(type(cosine_scores))
print(len(cosine_scores))
print(cosine_scores[0])

In [ ]:
# pearson 과 spearman 평균을 구함.
# => 실제 sts문장들 scores와 모델에서 구한 cosine_scores를 비교하여 Acc 평균 값들을 구함
from scipy.stats import pearsonr, spearmanr

eval_pearson_cosine, _ = pearsonr(scores, cosine_scores)
eval_spearman_cosine, _ = spearmanr(scores, cosine_scores)

print(eval_pearson_cosine)
print(eval_spearman_cosine)

In [ ]:
# 테스트 해봄
text = '난 널 사랑해'
inputs = tokenizer(text, return_tensors="pt")
out = tokenizer.tokenize(text)
print(out)

outputs = model(**inputs)
print(outputs)

output = outputs.last_hidden_state
print(output.shape)
#print(output[0][5])
onnx_embedding = output[0][5]
print(onnx_embedding)

In [ ]:
#sentence bert 원래 모델 로딩 
#model_checkpoint = "bongsoo/sentencebert_v1.2"
from sentence_transformers import SentenceTransformer, util
embedder = SentenceTransformer(model_checkpoint, device='cpu')
print(embedder)

In [ ]:
corpus = [
        '난 널 사랑해', 
        '난 매우 행복해', 
        '날씨가 좋다',
        '안 그래도 되는데 뭐. 괜찮아.',
        '내일 저녁까지 보수 공사가 끝날 것으로 예상합니다.',
        '감사합니다. 후회 없는 결정이 될 겁니다.',
        '그러면 오전 10시에 보도록 하죠. 내일 뵙겠습니다.',
        '프린트 카트리지가 다 떨어졌습니다. 더 주문할까요?',
        '내일 비가 온다',
        '오늘은 운동을 해야지',
        '내일은 다른팀과 축구경기가 있다',
        '주말에는 등산을 갈 예정이다'
        ]

In [ ]:
# ** 멀티로 한번에 tokenizer 할때는 반드시 padding=True 해야 함.(그래야 최대 길이 token에 맞춰서 padding 됨)
corpus_inputs = tokenizer(corpus, 
                 add_special_tokens=True, 
                 truncation=True, 
                 padding=True,   
                 max_length=128, 
                 return_tensors="pt")
print(corpus_inputs)
print(type(corpus_inputs))

In [ ]:
# onnx 양자화 모델 문장 유사도 출력 
import time

start = time.time()

# onnx 양자화 모델 문장 유사도 구함
outputs = model(**corpus_inputs)
last_hidden_state = outputs.last_hidden_state

print(last_hidden_state.shape)


# 첫번째 문장을 query로 지정함
in_mean_sequence = torch.mean(last_hidden_state[0], dim=0)

out_dict = {}
# for문을 돌면서 유사도 비교
for idx, hidden in enumerate(last_hidden_state):
    out_mean_sequence = torch.mean(hidden, dim=0)
    simul_score = util.pytorch_cos_sim(in_mean_sequence, out_mean_sequence)[0]
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
     # 사전 key로 순번으로 하고, 유사도를 저장함
    key = str(idx+1)
    out_dict[key] = simul_score
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
#print(out_dict)

# 사전 정렬(value(유사도)로 reverse=True 하여 내림차순으로 정렬함)
sorted_dict = sorted(out_dict.items(), key = lambda item: item[1], reverse = True)
#print(sorted_dict)

# 내립차순으로 정렬된 사전출력 
logger.info(f'---------------------------------------------------------')
logger.info(f'양자화/최적화된 모델 : {model_checkpoint}')
logger.info(f'---------------------------------------------------------')
for count in (sorted_dict):
    value = count[1].tolist() # count[1]은 2차원 tensor이므로 이를 list로 변환
    index = int(count[0])
    logger.info('{}, 유사도:{}'.format(corpus[index-1], value[0]))

logger.info(f'---------------------------------------------------------')
logger.info(f'=== 유사도 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

In [ ]:
# sentence bert 문장 유사도 출력 
import time

start = time.time()

# sentence bert 문장 임베딩값 구함
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
print(corpus_embeddings.shape)
#print(corpus_embeddings)

# 첫번째 문장을 query로 지정함
in_mean_sequence = corpus_embeddings[0]

out_dict = {}
# for문을 돌면서 유사도 비교
for idx, hidden in enumerate(corpus_embeddings):
    out_mean_sequence = hidden
    simul_score = util.pytorch_cos_sim(in_mean_sequence, out_mean_sequence)[0]
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
     # 사전 key로 순번으로 하고, 유사도를 저장함
    key = str(idx+1)
    out_dict[key] = simul_score
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
#print(out_dict)

# 사전 정렬(value(유사도)로 reverse=True 하여 내림차순으로 정렬함)
sorted_dict = sorted(out_dict.items(), key = lambda item: item[1], reverse = True)
#print(sorted_dict)

# 내립차순으로 정렬된 사전출력 
logger.info(f'---------------------------------------------------------')
logger.info(f'sbert모델: {model_checkpoint}')
logger.info(f'---------------------------------------------------------')
for count in (sorted_dict):
    value = count[1].tolist() # count[1]은 2차원 tensor이므로 이를 list로 변환
    index = int(count[0])
    logger.info('{}, 유사도:{}'.format(corpus[index-1], value[0]))

logger.info(f'---------------------------------------------------------')
logger.info(f'=== 유사도 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

In [ ]:
'''
# 참고
# => huggingface ORTModelForSequenceClassification 를 이용하여, ONNX 모델로 쉽게 저장할수 있다.(*단 여기서 ONNX 모델은 양자화된 모델은 아니고, 형식만 변경한 것임)
# => 양자화 하려면. 위에서 처럼 ORTQuantizer 이용해야 함.

from optimum.onnxruntime import ORTModelForSequenceClassification

model_checkpoint = "./distilbert-0331-TS-nli-0.1-10"

# Load model from hub and export it through the ONNX format 
model = ORTModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=3,
    from_transformers=True
)

# Save the exported model
model.save_pretrained("./onnxfolder")
'''

In [ ]:
'''
# onnx 모델 구조 로딩 해봄.
# => 맨 뒤에 return %last_hidden_state 리턴되면 => ORTModelForFeatureExtraction 사용 가능
import onnx
model = onnx.load("./distilbert-nli/model.onnx")
onnx.checker.check_model(model)
print(onnx.helper.printable_graph(model.graph))
'''